The purpose of this notebook is for testing the model by predicting the thusfar unseen dataset. But this could also be used for loading in any set of observations and bulk prediting them.

# _Loading in Model_

Choose your model from the ones created by _model_train_ or _warm_start_.